## **Datasets**

### Iris Dataset

Decision Tree for discretized dataset:

In [ ]:
from Dataset import Dataset
from DecisionTree import DecisionTree

irisDecisionTree = DecisionTree(Dataset().readCSV('datasets/', 'iris', True)) # Árvore do dataset (iris.csv)
irisDecisionTree.DFSPrint()

<petallength>
	1.4: Iris-setosa (12)
	1.3: Iris-setosa (7)
	1.5: Iris-setosa (14)
	1.7: Iris-setosa (4)
	1.6: Iris-setosa (7)
	1.1: Iris-setosa (1)
	1.2: Iris-setosa (2)
	1.0: Iris-setosa (1)
	1.9: Iris-setosa (2)
	4.7: Iris-versicolor (5)
	4.5:
		<sepallength>
			6.4: Iris-versicolor (1)
			5.7: Iris-versicolor (1)
			5.6: Iris-versicolor (1)
			6.2: Iris-versicolor (1)
			6.0: Iris-versicolor (2)
			5.4: Iris-versicolor (1)
			4.9: Iris-virginica (1)
	4.9:
		<sepalwidth>
			3.1: Iris-versicolor (1)
			2.5: Iris-versicolor (1)
			2.8: Iris-virginica (1)
			2.7: Iris-virginica (1)
			3.0: Iris-virginica (1)
	4.0: Iris-versicolor (5)
	4.6: Iris-versicolor (3)
	3.3: Iris-versicolor (2)
	3.9: Iris-versicolor (3)
	3.5: Iris-versicolor (2)
	4.2: Iris-versicolor (4)
	3.6: Iris-versicolor (1)
	4.4: Iris-versicolor (4)
	4.1: Iris-versicolor (3)
	4.8:
		<sepallength>
			5.9: Iris-versicolor (1)
			6.8: Iris-versicolor (1)
			6.2: Iris-virginica (1)
			6.0: Iris-virginica (1)
	4.3: Iris-versicol

Decision Tree for discretized dataset (for numerical values):

In [ ]:
from Dataset import Dataset
from DecisionTree import DecisionTree

irisDecisionTreeBinning = DecisionTree(Dataset().readCSV('datasets/', 'iris', True, True, None, 3), True) # Árvore do dataset (iris.csv) com os valores numéricos discretizados
irisDecisionTreeBinning.DFSPrint()

<petalwidth>
	0.10-0.90: Iris-setosa (50)
	0.90-1.70:
		<petallength>
			2.97-4.93: Iris-versicolor (47)
			4.93-6.90:
				<sepallength>
					5.50-6.70:
						<sepalwidth>
							2.00-2.80: Iris-virginica (3)
					6.70-7.90: Iris-virginica (1)
	1.70-2.50:
		<sepalwidth>
			2.80-3.60:
				<petallength>
					2.97-4.93:
						<sepallength>
							5.50-6.70: Iris-virginica (2)
					4.93-6.90:
						<sepallength>
							6.70-7.90: Iris-virginica (15)
							5.50-6.70: Iris-virginica (11)
			2.00-2.80: Iris-virginica (16)
			3.60-4.40: Iris-virginica (2)


## **GenDataset.py e GenDatasetBig.py** 

**GenDataset.py** 

In [ ]:
import sys
import os
import csv
import random
from copy import deepcopy


game_path = os.path.abspath(os.path.join(os.path.dirname(__file__), 'Game'))
sys.path.append(game_path)

from Game.searchAlgos.mcts import MCTS
from Game.fourGame import FourGame  

#função que gera o dataset
def gerar_dataset_csv(num_amostras=20000, path='datasets/connect4_dataset.csv', simbolo_ia='X'):
    with open(path, mode='w', newline='') as f:
        writer = csv.writer(f)

        #headers
        header = [f"cell_{row}_{col}" for row in range(6) for col in range(7)]
        header.append("piece_count")
        header.append("best_move")
        writer.writerow(header)

        exemplos_gerados = 0
        tentativas = 0

        while exemplos_gerados < num_amostras and tentativas < num_amostras * 10:
            tentativas += 1
            print(f"\n Tentativa #{tentativas}")

            game = FourGame(columns=7, lines=6)
            jogadas_totais = random.randint(6, 41)  # mid to late-game

            #varia o foco posicional: esquerda, centro, direita ou random extra
            pos_focus = random.choice(["left", "center", "right","random"])

            jogador = 'O' if simbolo_ia == 'X' else 'X'

            for i in range(jogadas_totais - 1):
                legal = game.getLegalMoves()
                if not legal or game.gameOver():
                    break

                #filtra jogadas com base no foco posicional
                if pos_focus == "left":
                    legal_focus = [col for col in legal if col <= 2]
                elif pos_focus == "center":
                    legal_focus = [col for col in legal if 2 <= col <= 4]
                elif pos_focus == "right":
                    legal_focus = [col for col in legal if col >= 4]
                #jogadas mais random
                else:
                    legal_focus = [col for col in legal if col <=6]

                if not legal_focus:
                    legal_focus = legal  

                move = random.choice(legal_focus)
                game.makeMove(move + 1, jogador)
                jogador = 'O' if jogador == 'X' else 'X'

            if game.gameOver():
                print(" Jogo terminou antes do MCTS.")
                continue

            mcts = MCTS(iaSymbol=simbolo_ia, game=deepcopy(game))
            mcts.search(0.5)
            best = mcts.bestMove()
            if not best:
                print("MCTS falhou.")
                continue

            best_move = best.move
            game.makeMove(best_move + 1, simbolo_ia)

            # cria entrada no CSV
            estado = []
            piece_count = 0
            for row in game.state:
                for cell in row:
                    if cell == 'X':
                        estado.append(1)
                        piece_count += 1
                    elif cell == 'O':
                        estado.append(2)
                        piece_count += 1
                    else:
                        estado.append(0)

            writer.writerow(estado + [piece_count, best_move])
            exemplos_gerados += 1
            print(f"Exemplo #{exemplos_gerados} gerado. Foco: {pos_focus}")

    print(f"\n{exemplos_gerados} exemplos gerados com sucesso em {path}")

if __name__ == "__main__":
   
    gerar_dataset_csv(num_amostras=20000)


**GenDatasetBig.py**  

In [ ]:
import sys
import os
import csv
import random
from copy import deepcopy

game_path = os.path.abspath(os.path.join(os.path.dirname(__file__), 'Game'))
sys.path.append(game_path)

from Game.searchAlgos.mcts import MCTS
from Game.fourGame import FourGame  

def gerar_dataset_csv(num_amostras=100000, path='datasets/connect4_dataset.csv', simbolo_ia='X'):
    with open(path, mode='w', newline='') as f:
        writer = csv.writer(f)

        #headers
        header = [f"cell_{row}_{col}" for row in range(6) for col in range(7)]
        header += ["piece_count", "first_player", "current_player", "best_move"]
        writer.writerow(header)

        exemplos_gerados = 0
        tentativas = 0

        while exemplos_gerados < num_amostras and tentativas < num_amostras * 10:
            tentativas += 1
            print(f"\nTentativa #{tentativas}")

            game = FourGame(columns=7, lines=6)
            jogadas_totais = random.randint(6, 41)  # de mid a late-game

            # Escolhe aleatoriamente o primeiro jogador
            first_player = random.choice(['X', 'O'])
            jogador = first_player if simbolo_ia == 'O' else ('O' if first_player == 'X' else 'X')

            # Varia o foco posicional: esquerda, centro, direita, aleatório
            pos_focus = random.choice(["left", "center", "right", "random"])

            for i in range(jogadas_totais - 1):
                legal = game.getLegalMoves()
                if not legal or game.gameOver():
                    break

                if pos_focus == "left":
                    legal_focus = [col for col in legal if col <= 2]
                elif pos_focus == "center":
                    legal_focus = [col for col in legal if 2 <= col <= 4]
                elif pos_focus == "right":
                    legal_focus = [col for col in legal if col >= 4]
                else:
                    legal_focus = legal

                if not legal_focus:
                    legal_focus = legal 

                move = random.choice(legal_focus)
                game.makeMove(move + 1, jogador)
                jogador = 'O' if jogador == 'X' else 'X'

            # Última jogada com MCTS 
            if game.gameOver():
                print("Jogo terminou antes do MCTS.")
                continue

            mcts = MCTS(iaSymbol=simbolo_ia, game=deepcopy(game))
            mcts.search(0.5)
            best = mcts.bestMove()
            if not best:
                print("MCTS falhou.")
                continue

            best_move = best.move
            game.makeMove(best_move + 1, simbolo_ia)
            current_player = simbolo_ia

            #cria entrada do CSV
            estado = []
            piece_count = 0
            for row in game.state:
                for cell in row:
                    if cell == 'X':
                        estado.append(1)
                        piece_count += 1
                    elif cell == 'O':
                        estado.append(2)
                        piece_count += 1
                    else:
                        estado.append(0)

            writer.writerow(estado + [piece_count, first_player, current_player, best_move])
            exemplos_gerados += 1
            print(f"Exemplo #{exemplos_gerados} gerado. Foco: {pos_focus}")

    print(f"\n{exemplos_gerados} exemplos gerados com sucesso em {path}")

if __name__ == "__main__":
    gerar_dataset_csv(num_amostras=100000)


Os códigos são identicos, a grande diferença são os atributos que contêm, a explicação está em "Connect4 Dataset" neste notebook.

Resumo:
    
    1. Escolhe um número aleatório de jogadas de 6 a 41
    2. Escolhe quem é o primeiro jogador e alterna entre jogadas
    3. Escolhe em que posições do tabuleiro que jogar mais, direita, esquerda , centro ou totalmente aleatório
    4. Faz jogadas com tudo em mente
    5. Na última jogada faz com o MCTS e obtêm-se o melhor move para o estado representado pelas células 

## **Estruturas de Dados**

### **Node**

In [ ]:
class Node():

    def __init__(self, attribute, value, label, isClass = False, counter = None):
        self.attribute = attribute
        self.isClass = isClass
        self.label = label
        self.value = value
        if (not isClass):
            self.neighbours = []
        else:
            self.counter = counter

    def addNeighbour(self, neighbour):
        self.neighbours.append(neighbour)
    
    def getNeighbours(self):
        return self.neighbours
        
    def getAttribute(self):
        return self.attribute

    def getValue(self):
        return self.value

#### Construtor da classe

In [ ]:
def __init__(self, attribute, value, label, isClass = False, counter = None):
    self.attribute = attribute
    self.isClass = isClass
    self.label = label
    self.value = value
    if (not isClass):
        self.neighbours = []
    else:
        self.counter = counter

Atributos:

- 'attribute': Representa o atributo pelo qual o nó é dividido. Se 'isClass' for 'True', este representa a classe final do nó.

- 'value': Representa o valor do nó.

- 'isClass': (Opcional, por omissão é False) Um valor booleano que nos indica se o nó representa uma classe final (folha).

- 'label': Nome do atributo em questão.

- 'neighbours' - Inicializa uma lista vazia para guardar os vizinhos (filhos) do nó, caso não seja um nó classe (folha).

- 'counter' - Número de exemplos do dataset que levam ao valor indicado pelo nó caso seja classe<br/><br/>

#### Métodos

Todos os métodos desta estrutura de dados são apenas getters da informação já anteriormente apresentada.

In [ ]:
def getNeighbours(self):
        return self.neighbours
        
def getAttribute(self):
    return self.attribute

def getValue(self):
    return self.value

No entanto, existe uma exceção no método 'addNeighbour':

In [ ]:
def addNeighbour(self, neighbour):
        self.neighbours.append(neighbour)

Este método permite-nos de facto fazer alterações no nó. Com o mesmo podemos adicionar nós vizinhos à lista criada no dado nó.<br/><br/>

## **DataSet**

A classe seguinte, permite-nos manipular os dados do dataset como uma matriz.

In [ ]:
import csv                                                                              
import copy                                                                             

class Dataset():

    def __init__(self, dataset=None, header=None):                                      
        if (dataset != None) and (header != None):                                      
            self.array = dataset                                                        
            self.header = header                                                        
            self.lines = len(self.array)                                                
            self.cols = len(self.array[0])                                              
    
    def readCSV(self, path, filename, hasId=False, hasHeader=True, headerInput=None):   
        csvFile = open(path + filename + '.csv', 'r')                                   
        reader = csv.reader(csvFile)                                                    

        if hasHeader:                                                                   
            self.header = next(reader)                                                  
            self.header.pop(0)                                                          
        else:
            self.header = headerInput                                                   

        self.array = []                                                                 
        for row in reader:                                                              
            self.array.append(row)                                                      

        if hasId:                                                                       
            for i in range(len(self.array)):                                            
                self.array[i].pop(0)                                                    

        self.lines = len(self.array)                                                    
        self.cols = len(self.array[0])                                                  
        return self                                                                     
    
    def getValue(self, line, col):                                                      
        return self.array[line][col]

    def copy(self):                                                                     
        return Dataset(copy.deepcopy(self.array), copy.deepcopy(self.header))
    
    def removeLine(self, line):                                                         
        self.array.pop(line)                                                            
        self.lines -= 1                                                                

    def removeColumn(self, col):                                                        
        if self.header: self.header.pop(col)                                            
        for i in range(len(self.array)):                                                
            self.array[i].pop(col)                                                     
        self.cols -= 1                                                                 

#### **Construtor da classe**

In [ ]:
def __init__(self, dataset = None, header = None):                                  # Inicializa a classe Dataset com dataset e header opcionais
    if ((dataset != None) and (header != None)):                                    # Verifica se dataset e header não são None
        self.array = dataset                                                        # Atribui dataset ao atributo array da instância
        self.header = header                                                        # Atribui header ao atributo header da instância
        self.lines = len(self.array)                                                # Calcula e armazena o número de linhas do dataset
        self.cols = len(self.array[0])                                              # Calcula e armazena o número de colunas do dataset

#### **Atributos:**
- 'self.array': Armazena os dados do dataset. 

- 'self.header': Armazena os nomes das colunas.

- 'self.lines': Armazena o número de linhas no dataset.

- 'self.cols': Armazena o número de colunas no dataset.

#### **Métodos**

O método 'readCSV':
- Lê um arquivo CSV e carrega os seus dados para a instância do Dataset.

In [ ]:
def readCSV(self, path, filename, hasId=False, hasHeader=True, headerInput = None, binCount = None): # Lê um arquivo CSV e carrega os dados no dataset
        csvFile = open(path+filename+'.csv', 'r')                                   # Abre o arquivo CSV no modo de leitura
        reader = csv.reader(csvFile)                                                # Cria um objeto reader para iterar sobre as linhas do CSV

        if (hasHeader == True):                                                     # Se o arquivo CSV tem um cabeçalho
            self.header = next(reader)                                              # Lê a primeira linha como cabeçalho
            self.header.pop(0)                                                      # Remove o primeiro elemento do cabeçalho (presumivelmente um ID)
        else:
            self.header = headerInput                                               # Se não houver cabeçalho, usa o headerInput fornecido

        self.array = []                                                             # Inicializa o array para armazenar os dados
        for row in reader:                                                          # Itera sobre as linhas restantes do CSV
            self.array.append(row)                                                  # Adiciona cada linha ao array

        if (hasId):                                                                 # Se as linhas têm um ID
            for i in range(len(self.array)):                                        # Itera sobre todas as linhas
                self.array[i].pop(0)                                                # Remove o primeiro elemento de cada linha (presumivelmente um ID)

        self.lines = len(self.array)                                                # Calcula e armazena o número de linhas do array
        self.cols = len(self.array[0])                                              # Calcula e armazena o número de colunas do array

        if (binCount != None):                                                      # Se binCount não for None                
            self.binning(binCount)                                                  # Chama a função binning com o valor de binCount
            
        return self                                                                 # Retorna a instância do dataset

O método 'getValue':
- Retorna o valor armazenado numa célula específica do dataset, identificada pelos índices da linha e coluna fornecidos.

In [ ]:
def getValue(self, line, col):                                                      # Retorna o valor na linha e coluna especificadas
        return self.array[line][col]

O método 'copy':
- Este método retorna uma cópia profunda da instância do Dataset, garantindo que alterações na cópia não afetem o original.

In [ ]:
def copy(self):                                                                     # Retorna uma cópia profunda do dataset
        return Dataset(copy.deepcopy(self.array), copy.deepcopy(self.header))

O método 'removeLine':
- Este método remove uma linha específica do dataset, identificada pelo índice fornecido.

In [ ]:
def removeLine(self, line):                                                         # Remove a linha especificada do dataset
        self.array.pop(line)                                                        # Remove a linha do array
        self.lines -= 1                                                             # Decrementa o contador de linhas

O método 'removeColumn':
- Este método remove uma coluna específica do dataset, identificada pelo índice fornecido. Se existir um cabeçalho, ele também é atualizado.

In [ ]:
def removeColumn(self, col):                                                        # Remove a coluna especificada do dataset
        if (self.header): self.header.pop(col)                                      # Se houver um cabeçalho, remove a coluna correspondente
        for i in range(len(self.array)):                                            # Itera sobre todas as linhas do array
            self.array[i].pop(col)                                                  # Remove a coluna de cada linha
        self.cols -= 1                                                              # Decrementa o contador de colunas

O método 'binning':
- Este método junta os valores númericos em "bin" (intervalos), diminuindo o tamanha do dataset mas levando a uma pequena perda de informação em alguns casos.

In [ ]:
def binning(self, binCount):                                                        # Bins continuous data into discrete intervals.
    for col in range(self.cols - 1):                                                # Assuming last column is the target
        try:                                                                        # Check if column is numeric
            colData = [float(self.array[i][col]) for i in range(self.lines)]        # Convert column data to float
        except ValueError:                                                          # If column is not numeric, skip
            continue                                                                

        minVal, maxVal = min(colData), max(colData)                                 # Get min and max values of column
        binWidth = (maxVal - minVal) / binCount                                     # Calculate bin width
        bins = [minVal + i * binWidth for i in range(binCount + 1)]                 # Create bins

        for i in range(self.lines):                                                 # Iterate through rows
            value = float(self.array[i][col])                                       # Get value of cell
            for b in range(len(bins) - 1):                                          # Iterate through bins
                if (bins[b] <= value < bins[b + 1]):                                # Check if value is within bin
                    self.array[i][col] = f"{bins[b]:.2f}-{bins[b + 1]:.2f}"         # Replace value with bin range
                    break                                                           # Exit loop
                else:                                                               # If value is not within bin
                    self.array[i][col] = f"{bins[-2]:.2f}-{bins[-1]:.2f}"           # Replace value with last bin range

## **Decision Tree Learning Algorithm: ID3**

O seguinte código define uma classe 'ID3', para a construção de uma árvore de decisão usando o algoritmo ID3. Este algoritmo é um método fundamental na classificação de dados e para a tomada de decisões. 


In [ ]:
import numpy as np

class ID3():  

    def __init__(self, dataset):
        self.dataset = dataset
        self.dataSetEntropy = self.__calcDatasetEntorpy()
        self.bestAtributte = self.__getBestGainAtributte()

    # Função de cálculo da entropia de determinado array com valores das classes 
    def __Entropy(self, X):
        sum = 0
        for i in X:
            if (X[i] == 1.0): return 0
            sum += -(X[i]) * np.log2(X[i])
        return sum
    
    # Função de cálculo da entropia do dataset
    def __calcDatasetEntorpy(self):
        values = {}
        for line in range(0, self.dataset.lines):
            value = self.dataset.getValue(line, self.dataset.cols-1)
            if (not (value in values)):
                values[value] = 1
            else:
                values[value] += 1

        for key in values:
            values[key] /= self.dataset.lines

        return self.__Entropy(values)
    
    # Função de decisão e escolha do melhor atributo do dataset apresentado
    def __getBestGainAtributte(self):
        maxGain = float('-inf')
        colMax = 0
        valuesMax = {}
        for j in range(0, self.dataset.cols-1):
            values = {}
            gain = self.dataSetEntropy
            for i in range(0, self.dataset.lines):
                value = self.dataset.getValue(i, j)

                if (not (value in values)):
                    values[value] = {"total": 0}
                
                classVar = self.dataset.getValue(i, self.dataset.cols-1)

                if (not (classVar in values[value])):
                    values[value][classVar] = 1
                else:
                    values[value][classVar] += 1
                
                values[value]["total"] += 1

            for key in values:
                for key2 in values[key]:
                    if key2 != "total":
                        values[key][key2] /= values[key]["total"]
                total = values[key].pop("total")
                gain -= (total/self.dataset.lines) * self.__Entropy(values[key])
                values[key]["total"] = total

            if (gain > maxGain):
                maxGain = gain
                colMax = j
                valuesMax = values

        return colMax, valuesMax

De seguida, iremos apresentar detalhadamente a classe e os métodos.<br/><br/>

### **Classe: 'ID3'**

A classe 'ID3' contém métodos para calcular a entropia de um dado conjunto de dados (dataset), determinar o melhor atributo para separar os dados, e inicializar a construção da árvore de decisão.<br/><br/>


#### **Construtor da classe**

In [ ]:
def __init__(self, dataset):
    self.dataset = dataset
    self.dataSetEntropy = self.__calcDatasetEntropy()
    self.bestAtributte = self.__getBestGainAttribute()

Atributos:

- 'dataset' - Guarda o dataset fornecido (Instância da classe Dataset).

- 'dataSetEntropy' - Guarda a entropia de todo o dataset, calculado pelo método '__calcDatasetEntropy'.

- 'bestAttribute' - Guarda o melhor atributo do dataset (ou seja o atributo ganho máximo), calculado por '__getBestGainAttribute'.<br/><br/>

#### **Método: Entropia**

Este método calcula a entropia do dataset, que mede a incerteza do mesmo.

In [ ]:
def __Entropy(self, X):
    sum = 0                                                                     # Inicializamos a soma com 0
    for i in X:                                                                 # Para cada elemento do dicionário X
        if (X[i] == 1.0): return 0                                              # Se o valor do elemento for 1, a entropia é 0
        sum += -(X[i]) * np.log2(X[i])                                          # Caso contrário, a soma fica com o simétrico do valor do elemento multiplicado pelo logaritmo de base 2 do valor do mesmo
    return sum                                                                  # Retornamos a soma


Parâmetros:

- 'X' - Um dicionário que representa a distribuição de frequência das classes.<br/><br/>

Retorno:

- Retorna a entropia do dataset.<br/><br/>

**Cálculo da entropia:**

A entropia é calculada segundo a seguinte fórmula:<br/><br/>

$$H(X) = -\sum p(x) \log_2 p(x)$$

Onde p(x) é a probabilidade da classe x.<br/><br/>

#### **Método: Cálculo da Entropia**

Este método calcula a entropia do dataset inteiro, primeiro determinando a distribuição de frequência das classes e depois aplicando a fórmula de cálculo da entropia (conforme apresentada anteriormente).

In [ ]:
def __calcDatasetEntropy(self):
    values = {}                                                                 # Dicionário para guardar a contagem de ocorrencias de cada classe
    for line in range(0, self.dataset.lines):                                   # Para cada linha do dataset                
        value = self.dataset.getValue(line, self.dataset.cols - 1)              # Obtemos o valor da classe
        if (not (value in values)):
            values[value] = 1                                                   # Se o valor ainda não estiver no dicionário, inicialziamos com o valor 1
        else:
            values[value] += 1                                                  # Caso já esteja no dicionário, incrementamos o valor em 1

    for key in values:                                                          # Para cada par chave-valor no dicionário                                           
        values[key] /= self.dataset.lines                                       # Calculamos a probabilidade de cada classe dividindo o número de ocorrencias pelo número total de linhas

    return self.__Entropy(values)                                               # Calculamos a entropia do dataset

Resumo:

1. É criado um dicionário 'values' vazio, onde iremos guardar a contagem de ocorrências de cada classe.

2. Percorremos cada linha do conjunto de dados, obtemos o valor de cada classe e atualizamos o dicionário 'values' para contar quantas vezes cada classe aparece no dataset.

Retorno:

- Retorna a entropia do dataset, invocando o método '__Entropy' apresentado anteriomente.<br/><br/>

#### **Método: Obter o atributo com maior ganho**

Este método determina qual o atributo (coluna) do dataset proporciona o maior ganho de informação. Este ganho é usado para decidir qual atributo usar para dividir os dados em cada nó da árvore de decisão.

In [ ]:
def __getBestGainAttribute(self):
    maxGain = float('-inf')                                                     # Inicializamos o ganho máximo com o menor valor possível, para garantir que qualquer ganho seja maior
    colMax = 0                                                                  # Inicializamos colmax a 0, que representa o índice da coluna com o maior ganho                         
    valuesMax = {}                                                              # Inicializamos valuesMax como um dicionário vazio, que guardará os valores de cada atributo da coluna com o maior ganho
    for j in range(0, self.dataset.colls - 1):                                  # Para cada coluna do dataset, exceto a última (que contém as classes)
        values = {}                                                             # Inicializamos um dicionário vazio para guardar os valores de cada valor do atributo
        gain = self.dataSetEntropy                                              # Inicializamos o ganho com a entropia do dataset
        for i in range(0, self.dataset.lines):                                  # Para cada linha do dataset
            value = self.dataset.getValue(i, j)                                 # Obtemos o valor do atributo na coluna j

            if (not (value in values)):                                         # Se o valor ainda não estiver no dicionário de valores                                  
                values[value] = {"total": 0}                                    # Inicializamos o valor no dicionário com um dicionário vazio, que guardará a contagem de cada classe

            classVar = self.dataset.getValue(i, self.dataset.colls - 1)         # Obtemos o valor da classe

            if (not (classVar in values[value])):                               # Se a classe ainda não estiver no dicionário do valor do atributo
                values[value][classVar] = 1                                     # Inicializamos a classe no dicionário do valor value com o valor 1
            else:                                                               # Caso contrário
                values[value][classVar] += 1                                    # Incrementamos o valor da classe no dicionário do valor value

            values[value]["total"] += 1                                         # Incrementamos o total de valores do atributo

        for key in values:                                                      # Para cada chave no dicionário de valores                            
            for key2 in values[key]:                                            # Para cada chave no dicionário de valores do atributo                      
                if key2 != "total":                                             # Se a chave não for "total"                            
                    values[key][key2] /= values[key]["total"]                   # Calculamos a probabilidade de cada classe dividindo o número de ocorrências pelo número total de valores do atributo
            total = values[key].pop("total")                                    # Removemos o total do dicionário de valores do atributo
            gain -= (total / self.dataset.lines) * self.__Entropy(values[key])  # Calculamos o ganho subtraindo a entropia do atributo multiplicada pela probabilidade do atributo
            values[key]["total"] = total                                        # Recuperamos o total do dicionário de valores do atributo

        if (gain > maxGain):                                                    # Se o ganho for maior que o ganho máximo
            maxGain = gain                                                      # Atualizamos o ganho máximo
            colMax = j                                                          # Atualizamos o índice da coluna com o maior ganho
            valuesMax = values                                                  # Atualizamos os valores do atributo da coluna com o maior ganho

    return colMax, valuesMax                                                    # Retornamos o índice da coluna com o maior ganho e os valores de cada atributo da coluna

Resumo:

1. Iteramos sobre cada atributo (coluna) do dataset.

2. Contamos as ocorrências de cada par (valor do atributo classe) e o total de linhas de cada valor do atributo (para calculo da probabilidade).

3. Calculamos o ganho de informação para o atributo.

4. Atualizamos o maior ganho e a melhor coluna, se necessário.

Retorno:

- Retorna um tuplo '(colMax, valuesMax)', onde o colMax representa o index do atributo (coluna) com o maior ganho de informação e valuesMax que são os diferentes valores do atributo e as suas frequências.<br/><br/>

## **Árvore de Decisão**

Já definidos o ID3 e a estrutura de dados Node, podemos por fim definir a Árvore de Decisão.

In [ ]:
from ID3 import ID3
from Node import Node
from Dataset import Dataset
import copy

class DecisionTree():

    def __init__(self, dataset, binning = False):
        self.initialDataset = dataset
        self.root = self.__generateNode(dataset)
        self.binning = binning

    def __generateNode(self, dataset, tabI=0, numRemovedColumns=0, value=None):
        attribute, values = ID3(dataset).bestAtributte
        node = Node(attribute, value, dataset.header[attribute])

        for key in values:
            isClass = False
            maxValue = float('-inf')
            maxkey = 'failed'
            maxkey2 = 'failed'
            for key2 in values[key]:
                if (key2 != "total"):
                    if (values[key][key2] == 1.0):
                        node.addNeighbour(Node(key2, key, None, True, values[key]["total"]))
                        isClass = True
                        break
                    elif (len(dataset.header) == 2):
                        if maxValue < values[key][key2] and key2 != "total":
                            maxValue = values[key][key2]
                            maxkey = key
                            maxkey2 = key2

            if (not isClass):
                if (len(dataset.header) == 2):
                    node.addNeighbour(Node(maxkey2, maxkey, None, True, int(values[maxkey]["total"]*values[maxkey][maxkey2])))
                else:
                    
                    datasetCopy = dataset.copy()
                    linestoRemove = []
                    for i in range(len(datasetCopy.array)):
                        if (datasetCopy.array[i][attribute] != key):
                            linestoRemove.append(i)

                    for x in sorted(linestoRemove, reverse=True):
                        datasetCopy.removeLine(x)

                    datasetCopy.removeColumn(attribute)

                    node.addNeighbour(self.__generateNode(datasetCopy, tabI+2, numRemovedColumns+1, key))

        return node

    def DFSPrint(self, tabI = 0, node = None):
        
        if (node == None):
            node = self.root
        
        print('\t'*tabI + '<'+node.label+'>')
        
        for currentNode in node.getNeighbours():

            if (currentNode.isClass):
                print(('\t'*(tabI+1))+currentNode.getValue()+': ' + currentNode.getAttribute() + ' (' + str(currentNode.counter) + ')')

            else:
                print(('\t'*(tabI+1))+currentNode.getValue()+':')
                self.DFSPrint(tabI+2, currentNode)

    def classifyMultipleExamples(self, path, file):

        dataset = Dataset().readCSV(path, file, True, False)
        for line in range(dataset.lines):
            classExmp = self.classifyExample(copy.deepcopy(dataset), line)
            if (classExmp == -1):
                print('Not Found!!')
            else:
                print('Line ' + str(line+1) + ' Class: ' + classExmp)
        return
    
    def classifyExample(self, dataset, line):

        actualNode = self.root
        value = dataset.array[line][actualNode.getAttribute()]

        while (actualNode.isClass != True):
            
            neighbours = actualNode.getNeighbours()
            
            found = False
            for node in (neighbours):
                if (self.binning == False):
                    if node.getValue() == value:
                        dataset.removeColumn(actualNode.getAttribute())
                        actualNode = node
                        if (actualNode.isClass != True): value = dataset.array[line][(actualNode.getAttribute())]
                        found = True
                        break
                else:
                    spliting = node.getValue().split('-')
                    if (spliting[0].replace(".", "", 1).isdigit() == True):
                        minVal = float(spliting[0])
                        maxVal = float(spliting[1])
                        value = float(value)
                        if value >= minVal and value <= maxVal:
                            dataset.removeColumn(actualNode.getAttribute())
                            actualNode = node
                            if (actualNode.isClass != True): value = dataset.array[line][(actualNode.getAttribute())]
                            found = True
                            break
                    else:
                        if node.getValue() == value:
                            dataset.removeColumn(actualNode.getAttribute())
                            actualNode = node
                            if (actualNode.isClass != True): value = dataset.array[line][(actualNode.getAttribute())]
                            found = True
                            break

            if (found == False): return -1
    
        return actualNode.getAttribute()

#### **Construtor da classe**

In [ ]:
def __init__(self, dataset, binning = False):
        self.initialDataset = dataset
        self.root = self.__generateNode(dataset)
        self.binning = binning                           

Atributos:

- 'initialDataset' - Guarda o dataset original (Instância da classe Dataset).
- 'root' - Armazena o nó raiz da árvore, gerado pela função '__generateNode'.
- 'binning' - Boolean que representa se foi ou não realizado binning nos valores numéricos (discretizing)

#### Método: Criar nós

O método faz uma construção recursiva da árvre de decisão, criando nós para os melhores atributos e nós filhos para os respetivos valores. Isto acontece até que todos os dados sejam classificados como folhas.

In [ ]:
def __generateNode(self, dataset, tabI=0, value=None):                          # Função recursiva para gerar os nós da árvore    
    attribute, values = ID3(dataset).bestAtributte                              # Obtemos o atributo com o maior ganho e seus valores
    node = Node(attribute, value)                                               # Criamos um nó com o atributo e o valor
    for key in values:                                                          # Para cada valor do atributo com o maior ganho
        isClass = False 
        maxValue = float('-inf')
        maxkey = 'failed'
        maxkey2 = 'failed'                                                        # Inicializamos a variável isClass como False
        for key2 in values[key]:                                                # Para cada chave no dicionário de valores do atributo
            if (key2 != "total"):                                               # Se a chave não for "total"
                if (values[key][key2] == 1.0):                                  # Se o valor da chave for 1
                    node.addNeighbour(Node(key2, key, True))                    # Adicionamos um nó com o valor da chave e o valor do atributo como True
                    isClass = True                                              # Atualizamos a variável isClass para True
                    break
                elif (len(dataset.header) == 2):                                # No caso dos valores terem sofrido binning existe uma pequena chance
                    if maxValue < values[key][key2] and key2 != "total":        # de o mesmo intrevalo ter duas classes então será escolhida a   
                        maxValue = values[key][key2]                            # de maior frequência
                        maxkey = key
                        maxkey2 = key2                                                    
        if (not isClass):
            if (len(dataset.header) == 2):
                node.addNeighbour(Node(maxkey2, maxkey, None, True, int(values[maxkey]["total"]*values[maxkey][maxkey2])))  # Se isClass for False  
            else:
                datasetCopy = dataset.copy()                                        # Copiamos o dataset
                linestoRemove = []                                                  # Inicializamos uma lista vazia para guardar as linhas a serem removidas
                for i in range(len(datasetCopy.array)):                             # Para cada linha do dataset
                    if (datasetCopy.array[i][attribute] != key):                    # Se o valor do atributo for diferente do valor do atributo com o maior ganho
                        linestoRemove.append(i)                                     # Adicionamos o índice da linha à lista de linhas a serem removidas

                for x in sorted(linestoRemove, reverse=True):                       # Para cada índice de linha na lista de linhas a serem removidas
                    datasetCopy.removeLine(x)                                       # Removemos a linha do dataset

                datasetCopy.removeCollum(attribute)                                 # Removemos a coluna do dataset

                node.addNeighbour(self.__generateNode(datasetCopy, tabI+2, key))    # Adicionamos um nó filho gerado recursivamente com o dataset cortado

    return node                                                                     # Retornamos o nó

Resumo:

1. Identificamos o melhor atributo para dividir os dados usando o algoritmo ID3.

2. Criamos um nó com esse atributo como raiz da subárvore.

3. Para cada valor do atributo, geramos nós filhos/folhas.

4. Os nós filhos representam ramificações da árvore com base nos próximos melhores atributos.<br/><br/>

Retorno:

- Retorna o nó raiz da árvore de decisão, que inclui recursivamente nós filhos para cada valor do melhor atributo identificado no dataset.<br/><br/>

#### **Método: Imprimir segundo um DFS**

O método imprime hierarquicamente a árvore de decisão, aplicando uma pesquisa em profundidade (DFS) na mesma.

In [ ]:
    
def DFSPrint(self, tabI=0, node=None):                                                             # Função para imprimir a árvore em profundidade
    
    if node is None:                                                                               # Função para imprimir a árvore em profundidade
        node = self.root                                                                           # O nó é a raiz da árvore

    print('\t' * tabI + '<' + str(node.label) + '>')                                               # Imprimimos o nome do atributo do nó
    
    for currentNode in node.getNeighbours():                                                        # Para cada nó vizinho do nó passado como parâmetro
        if currentNode.isClass:                                                                     # Se o nó for uma classe
            print('\t' * (tabI + 1) + str(currentNode.getValue()) + ': ' +                          # Imprimimos o valor do nó
                str(currentNode.getAttribute()) + ' (' + str(currentNode.counter) + ')')                
        else:                                                                                       # Caso contrário
            print('\t' * (tabI + 1) + str(currentNode.getValue()) + ':')                            # Imprimimos o valor do nó
            self.DFSPrint(tabI + 2, currentNode)                                                    # Chamamos a função recursivamente para imprimir os nós filhos


- Facilita a compreensão da estrutura da árvore de decisão.

- Permite uma visualização clara dos atributos e valores em cada nível da árvore.

## **Interface**
main.py

In [ ]:
from Dataset import Dataset
from DecisionTree import DecisionTree
from copy import deepcopy
import random

irisTree = DecisionTree(Dataset().readCSV('datasets/', 'iris', hasId=True))
irisTreeBinning = DecisionTree(Dataset().readCSV('datasets/', 'iris', hasId=True, binCount=3), binning=True)
connect4Tree = DecisionTree(Dataset().readCSV('datasets/', 'connect4_dataset', hasId=False, hasHeader=True))

def chooseTree():
    print("\nEscolhe uma das árvores:")
    print("[1] Iris")
    print("[2] Iris com Binning")
    print("[3] Connect 4")
    option = input("Opção: ")

    if option == '1':
        return irisTree, 'datasets/', 'iris', True
    elif option == '2':
        return irisTreeBinning, 'datasets/', 'iris', True
    elif option == '3':
        return connect4Tree, 'datasets/', 'connect4_dataset', False
    else:
        print("Opção inválida.")
        return chooseTree()

def main():
    while True:
        print("\nÁrvores de Decisão - Algoritmo ID3")
        print("1. Ver árvore")
        print("2. Classificar ficheiro CSV")
        print("3. Calcular precisão")
        print("4. Sair")

        op = input("Escolha a opção: ")

        if op == '1':
            tree, *_ = chooseTree()
            print("\nÁrvore:")
            tree.DFSPrint()

        elif op == '2':
            tree, path, file, hasId = chooseTree()
            dataset = Dataset().readCSV(path, file, hasId=hasId, hasHeader=True)

            print("\nClassificações:")
            for i in range(dataset.lines):
                predicted = tree.classifyExample(deepcopy(dataset), i)
                print(f"Linha {i + 1}: Classe prevista = {predicted}")

        elif op == '3':
            tree, path, file, hasId = chooseTree()
            dataset = Dataset().readCSV(path, file, hasId=hasId, hasHeader=True)
            random.shuffle(dataset.array)

            split_point = int(0.8 * dataset.lines)
            train_data = Dataset(deepcopy(dataset.array[:split_point]), deepcopy(dataset.header))
            test_data = Dataset(deepcopy(dataset.array[split_point:]), deepcopy(dataset.header))

            tree = DecisionTree(train_data)

            acertos = 0
            total = test_data.lines

            for i in range(total):
                previsto = tree.classifyExample(deepcopy(test_data), i)
                real = test_data.getValue(i, test_data.cols - 1)
                if previsto == real:
                    acertos += 1

            print(f"\nPrecisão no conjunto de teste: {acertos}/{total} = {acertos / total * 100:.2f}%")

        elif op == '4':
            print("Até à próxima!")
            break
        else:
            print("Opção inválida.")

if __name__ == "__main__":
    main()



Decision Trees using ID3 Algorithm!

Choose one action: 
 [1] Print 
 [2] Classify a csv file with examples 
 [3] Play Fourgame 
 [4] Leave
Invalid Input!

Decision Trees using ID3 Algorithm!

Choose one action: 
 [1] Print 
 [2] Classify a csv file with examples 
 [3] Play Fourgame 
 [4] Leave
Choose one tree: 
 [1] Weather Tenis Tree 
 [2] Weather Tenis Tree with Bining 
 [3] Restaurant Stay Tree 
 [4] Iris Tree 
 [5] Iris Tree with Bining 
 [6] Connect Four Tree (Can't be read or its hard to read)

Tree: 

<Weather>
	sunny:
		<Humidity>
			75.33-85.67: no (1)
			85.67-96.00: no (2)
			65.00-75.33: yes (2)
	overcast: yes (4)
	rainy:
		<Windy>
			FALSE: yes (3)
			TRUE: no (2)

Decision Trees using ID3 Algorithm!

Choose one action: 
 [1] Print 
 [2] Classify a csv file with examples 
 [3] Play Fourgame 
 [4] Leave
Choose one tree: 
 [1] Weather Tenis Tree 
 [2] Weather Tenis Tree with Bining 
 [3] Restaurant Stay Tree 
 [4] Iris Tree 
 [5] Iris Tree with Bining 
 [6] Connect Four Tr

## Connect4 Dataset

O dataset desenvolvido para o problema do Connect4 baseia-se em representar o estado atual do tabuleiro e prever a melhor próxima jogada, sendo esta determinada por meio de simulações com o algoritmo Monte Carlo Tree Search (MCTS).

Devido à elevada complexidade do jogo com mais de 4,5 triliões de possíveis estados distintos optou-se por adicionar três atributos complementares: first_player, current_player e piece_count.

first_player: indica quem iniciou a partida (X ou O),

current_player: indica quem está prestes a jogar (quem realizará a jogada recomendada pelo MCTS),

piece_count: representa o número total de peças já colocadas no tabuleiro até o momento.

A introdução destes atributos teve como principal objetivo reduzir a incerteza associada à previsão da jogada ideal, ao contextualizar melhor o estado atual da partida. Por exemplo, dois tabuleiros idênticos podem representar situações completamente diferentes consoante quem iniciou o jogo ou quem joga no momento atual.

No entanto, é importante salientar que a eficácia destes atributos está diretamente ligada à quantidade e à diversidade dos dados disponíveis:

Em datasets pequenos, a presença destes atributos pode conduzir a overfitting, dado que o modelo pode aprender padrões esparsos ou específicos demais.

Já em datasets maiores e mais balanceados, estes atributos oferecem uma melhor segmentação dos estados, permitindo à árvore de decisão realizar escolhas mais informadas e generalizáveis.

Assim, o seu uso é justificado em cenários com volumes substanciais de dados e serve para atenuar o impacto do desbalanceamento natural num jogo com espaço de estados tão vasto.

### Binning e valores numéricos nos Datasets

Valores numéricos levam a uma grande dimensão da árvore de decisão, no entanto são simples de ser agrupados em intervalos reduzindo o tamanho do dataset e consequentemente da árvore.

Uma das técnicas utilizadas é conhecida por binning que consiste em agrupar um conjunto de dados numéricos em intervalos, reduzindo o tamanho mas com o risco de perda de informação no processo quanto maior for o tamanho dos intervalos.

O objetivo então é encontrar o trade-off ideal de modo a reduzir o tamanho e manter a precisão o máximo possível.

Este algoritmo foi então aplicado ao dataset do weather e da iris apresentando algumas imprecisões no caso da iris no teste de exemplos dados.

Isto dá-se pelo fato de o mesmo intervalo ter precença de várias classes e sendo assim foi feita uma implementação que escolheria a de maior frequência reduzindo os erros.

No entanto como no exemplo em baixo em que o intervalo tem 3 Iris-virginica e 1 Iris-versicolor, oque significa que em 4 valores 1 irá ter um resultado errado.

![Teste](esquemas/intervalobinning.png "Title")

### Balanced Datasets

Existem duas defenições de datasets os balanceados e não balanceados. 

Os balanceados são aqueles em que a frequência das classes é igual entre si e os não balanceados em que essa mesma frequência não o é.

Datasets balanceados levam a árvores mais balanceadas e existem dois métodos para balanceamento dos datasets: Undersapling e Oversapling

Undersapling consiste em reduzir o número de exemplos das classes com maior frequência até que fique balanceado
Oversapling consiste em aumentar o número de exemplos das classes com menor frequência até que fique balanceado

No nosso caso o dataset da iris está balanceado tendo 33% de cada classe que contêm (50/50/50)